# Prepare excel read with pandas

In [1]:
import pandas as pd
# data = pd.read_excel(r'monthly_market_data - Copy.xlsx')
data = pd.read_excel(r'CRSP.xlsx')

# print(data)


# Read Excel to memnory and create classes

In [2]:
import operator

class CompanyInfo:
    def __init__(self, tickerKey, name, month,numberOfOutstandingShares, marketcap,b2m, adjClose,inv,op,sourceSubsectorCode
):
        self.tickerKey = tickerKey
        self.name = name
        self.month = month
        self.rollingAverage = None
        self.numberOfOutstandingShares = numberOfOutstandingShares if not pd.isnull(numberOfOutstandingShares) else None
        self.marketcap = marketcap if not pd.isnull(marketcap) else None
        self.b2m = b2m if not pd.isnull(b2m) else None
        self.closeAdjust = adjClose
        self.momentum = None
        self.sizeMOM = None
        self.sizeHML = None
        self.sizeRMW = None
        self.sizeCMA = None
        self.sizeLabel = None
        self.CMALabel = None
        self.RMWLabel = None
        self.B2MLabel = None
        self.currentYield = None
        self.inv = inv if not pd.isnull(inv) else None
        self.op = op if not pd.isnull(op) else None
        self.sourceSubsectorCode = sourceSubsectorCode


    def __repr__(self):
         return self.__str__()

    def __str__(self):
        return "tickerKey:"+str(self.tickerKey) + \
            "\t month:" + str(self.month) + \
            "\t closeAdjust:" + str(self.closeAdjust) + \
            "\t currentYield:" + str(self.currentYield) + \
            "\t numberOfOutstandingShares:" + str(self.numberOfOutstandingShares) + \
            "\t marketcap:" + str(self.marketcap) + \
            "\t b2m:" + str(self.b2m) + \
            "\t rollingAverage:" + str(self.rollingAverage) + \
            "\t momentum:" + str(self.momentum) + \
            "\t sizeLabel:" + str(self.sizeLabel) + \
            "\t sizeHML:" + str(self.sizeHML) + \
            "\t sizeRMW:" + str(self.sizeRMW) + \
            "\t sizeCMA:" + str(self.sizeCMA) +"\n"

    def convertToList(self):
      return [getattr(self,atr) for atr in self.__dict__]


companyInfoDict  = {}
def addCompanyInfoToList(companyInfo:CompanyInfo):
    if companyInfo.tickerKey not in companyInfoDict:
      companyInfoDict[companyInfo.tickerKey]={}
    if companyInfo.month not in companyInfoDict[companyInfo.tickerKey]:
      companyInfoDict[companyInfo.tickerKey][companyInfo.month]=companyInfo
    else:
      raise Exception("Company info exists." + str(companyInfo))

def findCompanyInfo(tickerKey, month):
    if tickerKey in companyInfoDict and month in companyInfoDict[tickerKey]:
      return companyInfoDict[tickerKey][month]
    return None

def displayCompanyInfoHeader():
  for tickerKey in companyInfoDict.keys():
      for month in companyInfoDict[tickerKey].keys():
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        attrs = []
        for attribute in vars(company):
          attrs.append(attribute)
        print(attrs)
        return

def displayCompanyInfo(tickerKeys:list,months:list):
    displayCompanyInfoHeader()
    for tickerKey in tickerKeys:
      sortedMonths = sorted(months)
      for month in sortedMonths:
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        print(company.convertToList())

allMonths = []


for i, companyInfoPandas in data.iterrows():
    ci = CompanyInfo(tickerKey=companyInfoPandas["TickerKey"],
                                       name=companyInfoPandas["TickerNamePooyaFA"],
                                       month=companyInfoPandas["DayKeyFA"]// 100,
                                       numberOfOutstandingShares=companyInfoPandas["NumberOfOutstandingShares"],
                                       marketcap=companyInfoPandas["marketcap"],
                                       b2m=companyInfoPandas["B2M"],
                                       adjClose=companyInfoPandas["AdjClose"],
                                       inv=companyInfoPandas["INV"],
                                       op=companyInfoPandas["OP"],
                                       sourceSubsectorCode=companyInfoPandas["SourceSubsectorCode"],
                                       )
    addCompanyInfoToList(ci)
    allMonths.append(ci.month)

allMonths = list(set(allMonths))
allMonths.sort()


In [3]:
# companies to excel
def exportCompanyListToExcel(excelFileName):
  rows = []
  for company in companyInfoDict.keys():
    for month in sorted(companyInfoDict[company].keys()):
      rows.append(companyInfoDict[company][month].convertToList())

  company_ = None
  for tickerKey in companyInfoDict.keys():
      for month in sorted(companyInfoDict[company].keys()):
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        company_ = company
        break
      if company_ is not None:
        break
  dfOut = pd.DataFrame(rows,columns=company_.__dict__)
  dfOut.to_excel(excelFileName+'.xlsx', sheet_name='TMPExport')

# Dict access sample and test

In [4]:
print(list(companyInfoDict.keys())[:10])
print(list(companyInfoDict[1].keys())[:10])
print(companyInfoDict[1][139712])


[1, 9, 10, 223, 234, 239, 243, 247, 251, 255]
[139712, 139801, 139802, 139803, 139804, 139805, 139806, 139807, 139808, 139809]
tickerKey:1	 month:139712	 closeAdjust:802.523299286721	 currentYield:None	 numberOfOutstandingShares:3000000000.0	 marketcap:4959000000000.0	 b2m:0.786263157894737	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None



# Extract company names

In [5]:
companyTickerSet = list(companyInfoDict.keys())
print(companyTickerSet)


[1, 9, 10, 223, 234, 239, 243, 247, 251, 255, 259, 271, 279, 283, 287, 291, 303, 315, 323, 347, 350, 383, 389, 395, 398, 403, 406, 411, 414, 422, 423, 430, 434, 438, 446, 447, 450, 454, 458, 463, 466, 474, 475, 478, 487, 490, 494, 499, 502, 510, 511, 515, 517, 526, 527, 534, 538, 550, 558, 563, 566, 574, 575, 578, 587, 591, 599, 603, 610, 615, 622, 627, 631, 635, 640, 646, 647, 655, 659, 667, 670, 680, 687, 689, 698, 708, 711, 718, 724, 725, 731, 733, 743, 744, 747, 754, 759, 762, 767, 772, 774, 779, 786, 787, 791, 798, 801, 820, 822, 827, 839, 842, 847, 852, 855, 863, 866, 871, 875, 879, 887, 890, 899, 902, 908, 909, 922, 931, 935, 939, 947, 951, 955, 968, 981, 990, 993, 1010, 1013, 1020, 1028, 1032, 1041, 1044, 1048, 1053, 1059, 1064, 1068, 1074, 1084, 1093, 1099, 1101, 1109, 1117, 1120, 1137, 1149, 1152, 1157, 1161, 1169, 1174, 1177, 1180, 1185, 1193, 1197, 1200, 1205, 1211, 1215, 1222, 1224, 1226, 1233, 1235, 1242, 1247, 1252, 1256, 1261, 1265, 1269, 1273, 1283, 1284, 1288, 1297, 1

# Extract company months

In [6]:
print(allMonths)
print("Total:",len(allMonths))

[138001, 138002, 138003, 138004, 138005, 138006, 138007, 138008, 138009, 138010, 138011, 138012, 138101, 138102, 138103, 138104, 138105, 138106, 138107, 138108, 138109, 138110, 138111, 138112, 138201, 138202, 138203, 138204, 138205, 138206, 138207, 138208, 138209, 138210, 138211, 138212, 138301, 138302, 138303, 138304, 138305, 138306, 138307, 138308, 138309, 138310, 138311, 138312, 138401, 138402, 138403, 138404, 138405, 138406, 138407, 138408, 138409, 138410, 138411, 138412, 138501, 138502, 138503, 138504, 138505, 138506, 138507, 138508, 138509, 138510, 138511, 138512, 138601, 138602, 138603, 138604, 138605, 138606, 138607, 138608, 138609, 138610, 138611, 138612, 138701, 138702, 138703, 138704, 138705, 138706, 138707, 138708, 138709, 138710, 138711, 138712, 138801, 138802, 138803, 138804, 138805, 138806, 138807, 138808, 138809, 138810, 138811, 138812, 138901, 138902, 138903, 138904, 138905, 138906, 138907, 138908, 138909, 138910, 138911, 138912, 139001, 139002, 139003, 139004, 139005,

In [7]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'sizeLabel', 'CMALabel', 'RMWLabel', 'B2MLabel', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, None, 10.2823076702193, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, None, 10.7442082100924, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, None, 11.0434394294015, None, None, None, None, None, None, None, None, None, None, None, None, 4424]


# Fill missing close adjust

In [8]:
def calculateCloseAdjust(tickerKey, monthIndex):
  privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])
  if privuseMonthCompanyInfo is None:
    return None
  nextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+1])

  if nextMonthCompanyInfo is not None:
    return ((nextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/2)) * privuseMonthCompanyInfo.closeAdjust

  if monthIndex + 2 >= len(allMonths):
    return None
  secondNextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+2])
  if secondNextMonthCompanyInfo is None:
    return None
  return ((secondNextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/3)) * privuseMonthCompanyInfo.closeAdjust
  
def calculateMarketcapAdjust(tickerKey, monthIndex):
  privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])
  if privuseMonthCompanyInfo is None or privuseMonthCompanyInfo.marketcap is None:
    return None
  nextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+1])

  if nextMonthCompanyInfo is not None and nextMonthCompanyInfo.marketcap is not None:
    return ((nextMonthCompanyInfo.marketcap / privuseMonthCompanyInfo.marketcap) ** (1/2)) * privuseMonthCompanyInfo.marketcap

  if monthIndex + 2 >= len(allMonths):
    return None
  secondNextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+2])
  if secondNextMonthCompanyInfo is None or secondNextMonthCompanyInfo.marketcap is None:
    return None
  return ((secondNextMonthCompanyInfo.marketcap / privuseMonthCompanyInfo.marketcap) ** (1/3)) * privuseMonthCompanyInfo.marketcap
  
  # print(calculatedCloseAdjust)

_missingData = []
for tickerKey in companyTickerSet:
  # companyHistoricalData = [companyInfo for companyInfo in companyInfoList if companyInfo.tickerKey == tickerKey]
  startOfData = False
  for monthIndex in range(1,len(allMonths)-1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is not None:
        startOfData = True
        continue
    if currentCompanyInfo is None:
      closeAdjust = calculateCloseAdjust(tickerKey,monthIndex)
      if closeAdjust is None: 
        if startOfData :
          _missingData.append(["closeAdjust for tick",tickerKey," Month",allMonths[monthIndex]," Is None"])
        continue
      privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])

      adjustedCompanyInfo = CompanyInfo(tickerKey,privuseMonthCompanyInfo.name,allMonths[monthIndex],None,None,None,None,None,None,privuseMonthCompanyInfo.sourceSubsectorCode)
      adjustedCompanyInfo.closeAdjust = closeAdjust
      # print(tickerKey,allMonths[month],privuseMonthCompanyInfo) # log before adjusted
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None privuse marketcap" ,privuseMonthCompanyInfo)
        continue
      adjustedCompanyInfo.marketcap = calculateMarketcapAdjust(tickerKey,monthIndex)
      if adjustedCompanyInfo.marketcap is None:
        print("***\tNo market cap while having closed adjust for TickerKey:" +  str(adjustedCompanyInfo.tickerKey) + " Month:" + str(adjustedCompanyInfo.month))
        continue
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None numberOfOutstandingShares" ,privuseMonthCompanyInfo)
        continue
      adjustedCompanyInfo.numberOfOutstandingShares = privuseMonthCompanyInfo.numberOfOutstandingShares
      addCompanyInfoToList(adjustedCompanyInfo)
      # print(tickerKey,allMonths[month]) # log adjusted


# print([companyInfo for companyInfo in _missingData if companyInfo[1] == 9])

None privuse marketcap tickerKey:2511	 month:138608	 closeAdjust:306.273415	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:1.62189583333333	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2511	 month:139004	 closeAdjust:281.955124	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:1.62189583333333	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2551	 month:138502	 closeAdjust:22.1411269068768	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2551	 month:138708	 closeAdjust:83.1643977955371	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 

In [9]:
print("{} of {}".format(len(_missingData),data.size))
print(_missingData[:10])

6914 of 873870
[['closeAdjust for tick', 9, ' Month', 138403, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138404, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138405, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139307, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139308, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139309, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138907, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138908, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138909, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138910, ' Is None']]


# Test data

In [10]:

displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'sizeLabel', 'CMALabel', 'RMWLabel', 'B2MLabel', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, None, 10.2823076702193, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, None, 10.7442082100924, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, None, 11.0434394294015, None, None, None, None, None, None, None, None, None, None, None, None, 4424]


# Yield calculations

In [11]:
maximumCurrentYield = 0.30
minimumCurrentYield = -0.30
for tickerKey in companyTickerSet:
  for monthIndex in range(1,len(allMonths)):
    lastMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex-1])
    if lastMonthData is None:
      continue
    thisMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if thisMonthData is None:
      continue
    thisMonthData.currentYield = max(minimumCurrentYield,min(thisMonthData.closeAdjust / lastMonthData.closeAdjust - 1,maximumCurrentYield))

In [12]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'sizeLabel', 'CMALabel', 'RMWLabel', 'B2MLabel', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, None, 10.2823076702193, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, None, 10.7442082100924, None, None, None, None, None, None, None, None, None, 0.04492187499999689, None, None, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, None, 11.0434394294015, None, None, None, None, None, None, None, None, None, 0.027850467289718184, None, None, 4424]


# Calculate rolling average

In [13]:
def getRollingDataWindow(tickerKey,rollingWindowSize,endMonthIndex):
  rollingDataWindow = []
  for monthIndex in range(endMonthIndex+1 - rollingWindowSize,endMonthIndex+1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is None:
        continue
    rollingDataWindow.append(currentCompanyInfo)
  return rollingDataWindow

In [14]:
def average(dataList):
    if len(dataList)==0:
        return None
    data = [info.currentYield for info in dataList]
    if None in data:
      return None
    # print(data)
    return sum(data) / len(dataList)

def valueWeightedAverage(dataList):
    if len(dataList)==0:
        return None
    lastMonth = allMonths[allMonths.index(dataList[0].month) - 1]
    data = [info.currentYield * findCompanyInfo(info.tickerKey,lastMonth).marketcap for info in dataList]
    marketcapSum = sum([findCompanyInfo(info.tickerKey,lastMonth).marketcap for info in dataList])
    if None in data:
      return None
    # print(data)
    return sum(data) / marketcapSum

# print(companyInfoList)
rollingAverageWindowSize = 6     #Can be change
for tickerKey in companyTickerSet:
    # print("tickerKey",tickerKey)
    for endMonthIndex in range(rollingAverageWindowSize-1,len(allMonths)):
      # print("endMonthIndex",endMonthIndex)
      rollingDataWindow = getRollingDataWindow(tickerKey, rollingAverageWindowSize,endMonthIndex)
      if len(rollingDataWindow) != rollingAverageWindowSize: continue
      currentCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey,allMonths[endMonthIndex])
      currentCompanyInfo.rollingAverage = average(rollingDataWindow)

In [15]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'sizeLabel', 'CMALabel', 'RMWLabel', 'B2MLabel', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, None, 10.2823076702193, None, None, None, None, None, None, None, None, None, None, None, None, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, None, 10.7442082100924, None, None, None, None, None, None, None, None, None, 0.04492187499999689, None, None, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, None, 11.0434394294015, None, None, None, None, None, None, None, None, None, 0.027850467289718184, None, None, 4424]


# set size

In [16]:
def getCompaniesInMonth(month):
  compainesInMonth = []
  for tickerKey in companyInfoDict.keys():
    if month in companyInfoDict[tickerKey]:
      compainesInMonth.append(companyInfoDict[tickerKey][month])
  return compainesInMonth

In [17]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeMOM = "Big"
    for company in companiesInMonth[median:]:
        company.sizeMOM = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeMOM ][-10:])

[]


In [18]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.marketcap is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeLabel = "Big"
    for company in companiesInMonth[median:]:
        company.sizeLabel = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeLabel ][-10:])

[]


In [19]:

    
    
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None]# and x.marketcap is not None
    # companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("rollingAverage"), reverse=True)
    count = math.floor(len(companiesInMonth) * 0.3) #It might be better to round it up instead of down to increase valid data
    for company in companiesInMonth[0:count]:
        company.momentum = "High"
    for company in companiesInMonth[len(companiesInMonth) - count:]:
        company.momentum = "Low"
    for company in companiesInMonth[count:len(companiesInMonth) - count]:
        company.momentum = "Notural"

print ([x for x in getCompaniesInMonth(allMonths[-2]) if x.momentum is not None ][-10:])

[tickerKey:101872	 month:140003	 closeAdjust:10080.0	 currentYield:0.1019041203760811	 numberOfOutstandingShares:2500000000.0	 marketcap:25050000000000.0	 b2m:0.118899219712526	 rollingAverage:-0.07669821603597977	 momentum:Low	 sizeLabel:Big	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:102007	 month:140003	 closeAdjust:23163.0	 currentYield:-0.2792195668409261	 numberOfOutstandingShares:300000000.0	 marketcap:7307100000000.0	 b2m:0.0417117011278196	 rollingAverage:-0.089974746832983	 momentum:Low	 sizeLabel:Small	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:102543	 month:140003	 closeAdjust:3660.0	 currentYield:-0.24380165289256195	 numberOfOutstandingShares:9358065000.0	 marketcap:33342785595000.0	 b2m:0.224278046895941	 rollingAverage:-0.10794014706127815	 momentum:Low	 sizeLabel:Big	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:102749	 month:140003	 closeAdjust:14340.0	 currentYield:-0.21265030472739255	 numberOfOutstandingShares:700000000.0	 marketcap:100

In [20]:
def avgTriple(first, second, third):
    avg = 0
    count = 0
    if first is not None:
        count += 1
        avg += first
    if second is not None:
        count += 1
        avg += second
    if third is not None:
        count += 1
        avg += third
    if count != 0:
        avg /= count
    return avg

# MomentomFactor factor (MOM)

In [21]:


MoMFactor = {}
MoMAverages = {}

for index,month in enumerate(allMonths[:-1]):
    # Get companies of corrent month
    companiesInMonth = getCompaniesInMonth(month)
    # Get companis for these filters SL SH BL BH
    SL = [company for company in companiesInMonth if company.sizeLabel == "Small" and company.momentum == "Low"]
    SN = [company for company in companiesInMonth if company.sizeLabel == "Small" and company.momentum == "Notural"]
    SH = [company for company in companiesInMonth if company.sizeLabel == "Small" and company.momentum == "High"]
    BL = [company for company in companiesInMonth if company.sizeLabel == "Big" and company.momentum == "Low"]
    BN = [company for company in companiesInMonth if company.sizeLabel == "Big" and company.momentum == "Notural"]
    BH = [company for company in companiesInMonth if company.sizeLabel == "Big" and company.momentum == "High"]
    # Get companies next month info
    BH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BH]
    BN_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BN]
    BL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BL]
    SH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SH]
    SN_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SN]
    SL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SL]
    # Get companies next month currentYield
    BH_average = valueWeightedAverage([x for x in BH_nextMonth if x is not None])
    BN_average = valueWeightedAverage([x for x in BN_nextMonth if x is not None])
    BL_average = valueWeightedAverage([x for x in BL_nextMonth if x is not None])
    SH_average = valueWeightedAverage([x for x in SH_nextMonth if x is not None])
    SN_average = valueWeightedAverage([x for x in SN_nextMonth if x is not None])
    SL_average = valueWeightedAverage([x for x in SL_nextMonth if x is not None])
    
    MoMAverages[month] = [SH_average,SN_average,SL_average,BH_average,BN_average,BL_average]
    MoMFactor[month]=avgTriple(SH_average, BH_average,None) - avgTriple(SL_average, BL_average,None)

print(MoMFactor)


{138001: 0, 138002: 0, 138003: 0, 138004: 0, 138005: 0, 138006: 0, 138007: 0.05763324477299721, 138008: 0.04171650601688936, 138009: 0.07063193711431613, 138010: 0.014822309237620195, 138011: 0.10087845602200095, 138012: 0.03985713470693855, 138101: 0.11759863563538962, 138102: 0.058535507703336875, 138103: 0.08923499798006251, 138104: 0.023577580583179637, 138105: -0.007246346028337293, 138106: -0.0248437344096891, 138107: 0.027398964068663267, 138108: 0.039341730388192886, 138109: 0.06937376285859458, 138110: 0.03568534972862195, 138111: 0.019634803663860632, 138112: 0.03968893737579864, 138201: 0.08189458500792064, 138202: 0.10004402815173809, 138203: 0.08888940967324097, 138204: 0.11284849856310533, 138205: -0.02373344194475606, 138206: -0.04817757567400953, 138207: 0.037357490495020504, 138208: 0.004631493533325787, 138209: 0.011010321021447878, 138210: 0.005873582195452386, 138211: -0.025127423689172337, 138212: -0.0018108884443295142, 138301: 0.019517804513878564, 138302: -0.043

# Book to market Calc

In [22]:
#  Fill all marketcap based on the last one
for companyTicker in companyTickerSet:
  for index,month in enumerate(allMonths):
    if month not in companyInfoDict[companyTicker]:
      firstData:CompanyInfo = companyInfoDict[companyTicker][list(companyInfoDict[companyTicker].keys())[0]]
      c = CompanyInfo(companyTicker,firstData.name,month,None,None,None,None,None,None, firstData.sourceSubsectorCode)
      if index > 0 and allMonths[index-1] in companyInfoDict[companyTicker]:
        c.marketcap = companyInfoDict[companyTicker][allMonths[index-1]].marketcap
        c.b2m = companyInfoDict[companyTicker][allMonths[index-1]].b2m
        c.inv = companyInfoDict[companyTicker][allMonths[index-1]].inv
        c.op = companyInfoDict[companyTicker][allMonths[index-1]].op
      companyInfoDict[companyTicker][month] = c
  # print(companyInfoDict[companyTicker][139712])

for month in sorted(companyInfoDict[1].keys())[:10]:
  print(companyInfoDict[1][month])

tickerKey:1	 month:138001	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138002	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138003	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138004	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeLabel:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138005	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage

In [23]:
# Accounting data fixed for 12 months
for companyTicker in companyTickerSet:
  lastB2MOfTheYear = None
  lastINVOfTheYear = None
  lastOpOfTheYear = None
  for month in allMonths:
    if month % 100 == 4:
      lastB2MOfTheYear = companyInfoDict[companyTicker][month].b2m
      lastINVOfTheYear = companyInfoDict[companyTicker][month].inv
      lastOpOfTheYear = companyInfoDict[companyTicker][month].op
    companyInfoDict[companyTicker][month].b2m = lastB2MOfTheYear     
    companyInfoDict[companyTicker][month].inv = lastINVOfTheYear     
    companyInfoDict[companyTicker][month].op = lastOpOfTheYear     

In [24]:
# for month in sorted(companyInfoDict[1].keys())[:10]:
#   print(companyInfoDict[9][month])

In [25]:
# Book To market ranking

for index,month in enumerate(allMonths[:-1]):
  b2mMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None :
      # companyInfoDict[companyTicker][month].marketcap == None or \
      # findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue
    b2mMonthList.append(companyInfoDict[companyTicker][month].b2m)
  if len(b2mMonthList) == 0 :
    continue
  b2mMonthList.sort()
  lowBookToMarket = b2mMonthList[int(len(b2mMonthList)*0.3)-1]
  highBookToMarket = b2mMonthList[-int(len(b2mMonthList)*0.3)]

  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None:
      continue
    # if companyInfoDict[companyTicker][month].currentYield == None:
    #   continue
    if companyInfoDict[companyTicker][month].b2m <= lowBookToMarket:
      companyInfoDict[companyTicker][month].B2MLabel = "Low"
    elif companyInfoDict[companyTicker][month].b2m >= highBookToMarket:
      companyInfoDict[companyTicker][month].B2MLabel = "High"
    else:
      companyInfoDict[companyTicker][month].B2MLabel = "Neutral"



In [26]:
# RMW (Operational profit) ranking

for index,month in enumerate(allMonths[:-1]):
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None :
      # companyInfoDict[companyTicker][month].marketcap == None or \
      # findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue
    dataMonthList.append(companyInfoDict[companyTicker][month].op)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)-1]
  robustValue = dataMonthList[-int(len(dataMonthList)*0.3)]
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None:
      continue
    # if companyInfoDict[companyTicker][month].currentYield == None:
    #   continue
    if companyInfoDict[companyTicker][month].op <= weakValue:
      companyInfoDict[companyTicker][month].RMWLabel = "Weak"
    elif companyInfoDict[companyTicker][month].op >= robustValue:
      companyInfoDict[companyTicker][month].RMWLabel = "Robust"
    else:
      companyInfoDict[companyTicker][month].RMWLabel = "Neutral"

In [27]:

for companyTicker in companyTickerSet[:10]:
  print(companyInfoDict[companyTicker][139807].RMWLabel)

None
Robust
Robust
Neutral
Robust
Weak
Robust
Robust
Robust
Neutral


In [28]:
# CMA (Investment) ranking

for index,month in enumerate(allMonths[:-1]):
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None :
      # companyInfoDict[companyTicker][month].marketcap == None or \
      # findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue   
    dataMonthList.append(companyInfoDict[companyTicker][month].inv)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)-1]
  robustValue = dataMonthList[-int(len(dataMonthList)*0.3)]
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None:
      continue
    # if companyInfoDict[companyTicker][month].currentYield == None:
    #   continue
    if companyInfoDict[companyTicker][month].inv <= weakValue:
      companyInfoDict[companyTicker][month].CMALabel = "Conservative"
    elif companyInfoDict[companyTicker][month].inv >= robustValue:
      companyInfoDict[companyTicker][month].CMALabel = "Aggressive"
    else:
      companyInfoDict[companyTicker][month].CMALabel = "Neutral"

In [29]:
def getNextMonth(currentMonth):
  return allMonths[allMonths.index(currentMonth)+1]

def getNextMonthCompaniesHavingYeild(companyInfoList):
  response = []
  for companyInfo in companyInfoList:
    nextMoonth = companyInfoDict[companyInfo.tickerKey][getNextMonth(companyInfo.month)]
    if nextMoonth.currentYield != None:
      response.append(nextMoonth)
  return response


In [30]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.b2m is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeHML = "Big"
    for company in companiesInMonth[median:]:
        company.sizeHML = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeHML ][-10:])

[]


In [31]:
HMLFactor = {}
HMLAverages = {}
SMBBTMFactor = {}
for month in allMonths[:-1]:
  SH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "High" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "Low" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  BH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "High" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].B2MLabel == "Low" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  
  
  SH_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SH))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SL_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SL))
  BH_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BH))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BL_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BL))
  
  HMLAverages[month] = [SH_average,SN_average,SL_average,BH_average,BN_average,BL_average]
  SMBBTMFactor[month] = avgTriple(SH_average,SN_average,SL_average) - avgTriple(BH_average, BN_average, BL_average)
  HMLFactor[month] = avgTriple(SH_average, BH_average,None)  - avgTriple(SL_average , BL_average,None)
for month in list(HMLFactor.keys())[:10]:
  print(HMLFactor[month])

0
0
0
-0.014184660851324735
0.21666869349229742
-0.14980530123038793
-0.14482780900691594
-0.15167021928928115
-0.07913669064748174
-0.08186666666666653


In [32]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.op is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeRMW = "Big"
    for company in companiesInMonth[median:]:
        company.sizeRMW = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeRMW ][-10:])

[]


In [33]:
RMWFactor = {}
RMWAverages = {}
SMBRMWFactor = {}
for month in allMonths[:-1]:
  SR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Robust" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Weak" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  BR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Robust" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWLabel == "Weak" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  
  SR_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SR))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SW_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SW))
  BR_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BR))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BW_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BW))

  RMWAverages[month] = [SR_average,SN_average,SW_average,BR_average,BN_average,BW_average]
  SMBRMWFactor[month] = avgTriple(SR_average , SN_average, SW_average) - avgTriple(BR_average, BN_average, BW_average)
  RMWFactor[month] = avgTriple(SR_average, BR_average,None) - avgTriple(SW_average, BW_average,None)
for month in list(RMWFactor.keys())[:10]:
  print(RMWFactor[month])

0
0
0
0
0
0
0
0
0
0


In [34]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.op is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeCMA = "Big"
    for company in companiesInMonth[median:]:
        company.sizeCMA = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeCMA ][-10:])

[]


In [35]:
CMAFactor = {}
CMAAverages = {}
SMBCMAFactor = {}
for month in allMonths[:-1]:
  SA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Aggressive" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  SC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Conservative" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Small"]
  BA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Aggressive" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  BC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMALabel == "Conservative" and \
        companyInfoDict[companyTicker][month].sizeLabel == "Big"]
  
  SA_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SA))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SC_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SC))
  BA_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BA))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BC_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BC))

  CMAAverages[month] = [SA_average,SN_average,SC_average,BA_average,BN_average,BC_average]
  SMBCMAFactor[month] = avgTriple(SA_average, SN_average, SC_average) - avgTriple(BA_average, BN_average, BC_average)
  CMAFactor[month] = avgTriple(SC_average, BC_average,None)  - avgTriple(SA_average, BA_average,None)
for month in list(CMAFactor.keys())[:10]:
  print(CMAFactor[month])

0
0
0
0
0
0
0
0
0
0


In [36]:
SMBFactor = {}
for month in SMBBTMFactor.keys():
  SMBFactor[month] = avgTriple(SMBBTMFactor[month], SMBCMAFactor[month], SMBRMWFactor[month])


# Export company list to excel

In [37]:
# Example to excel function
exportCompanyListToExcel("calculations")

In [38]:
# Factors to excel
rows = []
for month in allMonths:
    data_ = [month,
      SMBFactor[month] if month in SMBFactor else None,
      HMLFactor[month] if month in HMLFactor else None]
    for i in range(6):
      data_.append(HMLAverages[month][i] if month in HMLAverages else None)
    
    data_.append(RMWFactor[month] if month in RMWFactor else None)
    for i in range(6):
      data_.append(RMWAverages[month][i] if month in RMWAverages else None)
    
    data_.append(CMAFactor[month] if month in CMAFactor else None)
    for i in range(6):
      data_.append(CMAAverages[month][i] if month in CMAAverages else None)
      
    data_.append(MoMFactor[month] if month in MoMFactor else None)
    for i in range(6):
      data_.append(MoMAverages[month][i] if month in MoMAverages else None)
      
    rows.append(data_)

dfOut = pd.DataFrame(rows, columns=['month',"SMBFactor", "HMLFactor","SH_average","SN_average","SL_average","BH_average","BN_average","BL_average", "RMWFactor","SR_average","SN_average","SW_average","BR_average","BN_average","BW_average", "CMAFactor","SA_average","SN_average","SC_average","BA_average","BN_average","BC_average", "MoMFactor","SH_average","SN_average","SL_average","BH_average","BN_average","BL_average"])
dfOut.to_excel('Factors.xlsx', sheet_name='Factors')